In [1]:
import pandas as pd
from pathlib import Path
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panag\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [104]:
# Set the absolute path to your data directory
data_directory = Path("E:/panag/Desktop/Ms Data Science/6 Quarter/Data Science Challenge/data_challenge_aueb_2023")

# Train data
y_train_file = "y_train.txt"
# Authors data
authors_file = "authors.txt"
# Abstract data
abstract_file = "abstract.txt"
# Test data
test_file = "test.txt"
# Year data
year_file = "year.txt"

y_train_path  = data_directory / y_train_file
authors_path  = data_directory / authors_file
abstract_path = data_directory / abstract_file
test_path     = data_directory / test_file
year_path     = data_directory / year_file

### Algorithm

In [105]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV

In [106]:
# Read training data
train_papers = list()
y_train = list()
with open(y_train_path, "r") as f:
    for line in f:
        t = line.split(',')
        train_papers.append(int(t[0]))
        y_train.append(t[1][:-1])

# Read test data
test_papers = list()
with open(test_path, "r") as f:
    for line in f:
        t = line.split(',')
        test_papers.append(int(t[0]))

In [139]:
#### All papers ####

# Import Numeric features
df_edgelist = pd.read_csv('features_edgelist.csv')
df_authors  = pd.read_csv('features_authors.csv')

df_year = pd.read_csv(year_path, header=None)
df_year.columns = ['paper_id','year']

# Numeric Features (Edgelist and Year)
df_numeric = pd.merge(df_edgelist,df_year, on='paper_id')

# Text feautures
df_text = pd.read_csv('cleaned_abstracts.csv')

In [155]:
#####  Train data #####
y_train = pd.read_csv(y_train_path, header=None)
y_train.columns = ['paper_id','class']

# merge df1 with df2
merged_df = pd.merge(y_train, df_text, on="paper_id", how="inner")

# merge the resulting dataframe with df3
df_train = pd.merge(merged_df, df_numeric, on="paper_id", how="inner")

df_train

,paper_id,class,abstract,links,classes,class0,class1,class2,class3,class4,class0_weight,class1_weight,class2_weight,class3_weight,class4_weight,year
0,2,4,The CFG recognition problem is: given a contex...,"[132575, 36628, 21383, 103558, 103665, 99219, ...","[2, 4, 4, 4, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]",0,0,2,0,13,0.000000,0.000000,0.133333,0.000000,0.866667,2015
1,3,3,to organize and present search results plays a...,"[158930, 20562, 36447, 157194, 16449, 97870, 1...","[3, 0, 3]",1,0,0,2,0,0.333333,0.000000,0.000000,0.666667,0.000000,2011
2,6,0,Modern applications in sensitive domains such ...,"[85084, 26965, 125973, 145744, 162490, 82406, ...","[0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, 1]",10,1,0,2,0,0.769231,0.076923,0.000000,0.153846,0.000000,2014
3,8,0,"We present discriminative Gaifman models, a no...","[107143, 22820, 63108, 52816, 143269, 140820, ...","[0, 0, 0, 2, 0, 2, 2, 0]",5,0,3,0,0,0.625000,0.000000,0.375000,0.000000,0.000000,2016
4,13,4,NaN,"[49261, 155149, 166781, 119572, 111639, 29039,...",[4],0,0,0,0,1,0.000000,0.000000,0.000000,0.000000,1.000000,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35158,166969,1,Multi-modal clustering aims to cluster data in...,"[53079, 37983, 145861, 10674, 25655, 28153, 27...","[1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1]",7,6,0,0,0,0.538462,0.461538,0.000000,0.000000,0.000000,2020
35159,166976,2,We present an NLP system that classifies the a...,"[46739, 76333, 11345, 33362, 100141, 58974, 12...",[],0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2011
35160,166977,1,learning researchers have a keen interest in p...,"[67159, 121767, 40215, 24436, 84137, 55653]",[],0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,2022
35161,166978,3,"Novelty, coverage and balance are important re...","[74571, 152542, 88474, 133478, 9403, 148867, 2...","[3, 0]",1,0,0,1,0,0.500000,0.000000,0.000000,0.500000,0.000000,2010


In [140]:
df_all = pd.merge(df_text, df_numeric, left_index=True, right_index=True)

# Split in train/test dataframe
mask_train = df_all.index.isin(train_papers)
mask_test = df_all.index.isin(test_papers)

df_train = df_all.loc[mask_train]
df_test = df_all.loc[mask_test]

In [144]:
df_train.isnull().sum(axis = 0)

paper_id_x          0
abstract         1920
paper_id_y          0
links               0
classes             0
class0              0
class1              0
class2              0
class3              0
class4              0
class0_weight    4443
class1_weight    4443
class2_weight    4443
class3_weight    4443
class4_weight    4443
year                0
dtype: int64

In [ ]:
####  Fill NaN values  ####

# Numeric Features
df_numeric.fillna(0.2, inplace=True)

# Text Features  ----  In text features fill na values per class
df_text['abstract'].fillna('', inplace=True)

In [137]:
df_all = pd.merge(df_text, df_numeric, left_index=True, right_index=True)

# Split in train/test dataframe
mask_train = df_all.index.isin(train_papers)
mask_test = df_all.index.isin(test_papers)

df_train = df_all.loc[mask_train]
df_test = df_all.loc[mask_test]

In [124]:
class NumericFeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.columns].values

In [125]:
class TextFeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]

In [126]:
# Assuming df_train is your dataframe and y_train is your target variable
numeric_columns = [
    'class0_weight', 'class1_weight', 'class2_weight', 'class3_weight', 'class4_weight','year' 
]

In [127]:
# Create a FeatureUnion object
feature_union = FeatureUnion(
    transformer_list=[
        ('tfidf', Pipeline([
            ('text_selector', TextFeaturesExtractor(column='abstract')),
            ('vectorizer', TfidfVectorizer(analyzer = 'word', stop_words='english'))
        ])),  # Text features
        ('numeric', NumericFeaturesExtractor(columns=numeric_columns))  # Numeric features
    ]
)

# Create a pipeline with FeatureUnion and LightGBM
pipeline = Pipeline(
    steps=[
        ('feature_union', feature_union),
        ('classifier', LGBMClassifier(objective='multiclass', num_classes=5))
    ]
)

#### Without Randomized Search

In [128]:
# Fit the pipeline to your data
pipeline.fit(df_train, y_train)

Pipeline(steps=[('feature_union',
                 FeatureUnion(transformer_list=[('tfidf',
                                                 Pipeline(steps=[('text_selector',
                                                                  TextFeaturesExtractor(column='abstract')),
                                                                 ('vectorizer',
                                                                  TfidfVectorizer(stop_words='english'))])),
                                                ('numeric',
                                                 NumericFeaturesExtractor(columns=['class0_weight',
                                                                                   'class1_weight',
                                                                                   'class2_weight',
                                                                                   'class3_weight',
                                                                                   'class4_weight',
                                                                                   'year']))])),
                ('classifier',
                 LGBMClassifier(num_classes=5, objective='multiclass'))])

In [131]:
# Predict probabilities for each class
y_pred = pipeline.predict_proba(df_test)

In [132]:
# Write predictions to a file
with open('sample_submission.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = list()
    for i in range(5):
        lst.append('class_'+str(i))
    lst.insert(0, "paperID")
    writer.writerow(lst)
    for i,test_paper in enumerate(test_papers):
        lst = y_pred[i,:].tolist()
        lst.insert(0, test_paper)
        writer.writerow(lst)

#### With Randomized Search

In [58]:
param_dist = {
    'feature_union__tfidf__vectorizer__max_df': [0.5, 0.75, 1.0],
    'classifier__n_estimators': [50, 100, 150, 200],
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2],
    'classifier__num_leaves': [30, 50, 100],
    'classifier__min_child_samples': [10, 20, 30],
}

random_search = RandomizedSearchCV(
    estimator = pipeline,
    param_distributions=param_dist,
    n_iter=5,     # Number of random combinations to try
    cv=3,          # Number of cross-validation folds
    n_jobs=-1,     # Use all available CPU cores
    verbose=10,
    random_state=42,
)

In [59]:
#### Use Random Search
random_search.fit(df_train, y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


RandomizedSearchCV(cv=3,
                   estimator=Pipeline(steps=[('feature_union',
                                              FeatureUnion(transformer_list=[('tfidf',
                                                                              Pipeline(steps=[('text_selector',
                                                                                               TextFeaturesExtractor(column='abstract')),
                                                                                              ('vectorizer',
                                                                                               TfidfVectorizer(stop_words='english'))])),
                                                                             ('numeric',
                                                                              NumericFeaturesExtractor(columns=['class0_weight',
                                                                                                                'class1_weight',
                                                                                                                'class2_weight',
                                                                                                                'class3_weight',
                                                                                                                'cl...
                                              LGBMClassifier(num_classes=5,
                                                             objective='multiclass'))]),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'classifier__learning_rate': [0.01,
                                                                      0.05, 0.1,
                                                                      0.2],
                                        'classifier__min_child_samples': [10,
                                                                          20,
                                                                          30],
                                        'classifier__n_estimators': [50, 100,
                                                                     150, 200],
                                        'classifier__num_leaves': [30, 50, 100],
                                        'feature_union__tfidf__vectorizer__max_df': [0.5,
                                                                                     0.75,
                                                                                     1.0]},
                   random_state=42, verbose=10)

In [60]:
print("Best parameters found: ", random_search.best_params_)
print("Best score found: ", random_search.best_score_)

Best parameters found:  {'feature_union__tfidf__vectorizer__max_df': 0.75, 'classifier__num_leaves': 30, 'classifier__n_estimators': 200, 'classifier__min_child_samples': 30, 'classifier__learning_rate': 0.2}
Best score found:  0.9000654096635667


In [61]:
# Predict probabilities for each class using the best estimator
best_estimator = random_search.best_estimator_
y_pred = best_estimator.predict_proba(df_test)

In [62]:
# Write predictions to a file
with open('sample_submission_random.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = list()
    for i in range(5):
        lst.append('class_'+str(i))
    lst.insert(0, "paperID")
    writer.writerow(lst)
    for i,test_paper in enumerate(test_papers):
        lst = y_pred[i,:].tolist()
        lst.insert(0, test_paper)
        writer.writerow(lst)